In [84]:
import fitz  # pyMuPDF
import re
import csv
import os
from pathlib import Path
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [85]:
def extract_text_from_pdf(pdf_path):
    """PDF 파일에서 텍스트를 추출합니다."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text
    except Exception as e:
        logger.error(f"PDF 파일 읽기 오류 ({pdf_path}): {e}")
        return None

def extract_date_from_filename(filename):
    """파일명에서 날짜 정보를 추출합니다."""
    patterns = [
        r'(\d{4})년도?\s*(\d{1,2})월',
        r'(\d{4})-(\d{1,2})',
        r'(\d{4})\.(\d{1,2})',
        r'(\d{4})_(\d{1,2})'
    ]
    
    for pattern in patterns:
        date_match = re.search(pattern, filename)
        if date_match:
            year = date_match.group(1)
            month = date_match.group(2).zfill(2)
            return f"{year}-{month}-01"
    
    return "날짜 정보 없음"

def clean_line(line):
    """라인을 정리합니다."""
    # 다양한 유니코드 공백 문자 제거
    line = re.sub(r'[\u00a0\u2002\u2003\u2009\u3000]', ' ', line)
    # 연속된 공백을 하나로 통합
    line = re.sub(r'\s+', ' ', line.strip())
    return line

def merge_split_chapter_lines(lines):
    """'제1장', '총', '칙'처럼 줄이 쪼개진 라인들을 병합합니다."""
    merged_lines = []
    i = 0
    while i < len(lines):
        line = clean_line(lines[i])

        # '제1장' 단독 라인을 만나면 병합 시작
        if re.match(r'^제\s*\d+\s*장$', line):
            merged = [line]
            j = i + 1
            # 다음 두 줄까지 병합 시도
            while j < len(lines):
                next_line = clean_line(lines[j])
                # 비어 있거나 구조적 키워드 만나면 중단
                if not next_line or re.match(r'^제\s*\d+\s*조', next_line) or re.match(r'^제\s*\d+\s*절', next_line) or re.match(r'^제\s*\d+\s*장', next_line) or re.match(r'^부\s*칙', next_line):
                    break
                merged.append(next_line)
                j += 1
            merged_lines.append(' '.join(merged))
            i = j
        else:
            merged_lines.append(line)
            i += 1

    return merged_lines


In [86]:
def extract_chapter_title(text):
    """장 제목을 정리하여 추출합니다."""
    if not text:
        return ""
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # 특수 문자 및 불필요한 공백 정리
    text = re.sub(r'[<>{}]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def parse_legal_document(filename, content):
    """법률 문서를 파싱하여 구조화된 데이터로 변환합니다."""
    last_modified = extract_date_from_filename(filename)
    
    parsed_data = []
    current_chapter = ""
    current_chapter_title = ""
    current_section = ""
    current_section_title = ""
    current_article = ""
    current_article_title = ""
    current_content = []
    article_id = 0
    
    # 장 제목 수집을 위한 버퍼
    chapter_buffer = []
    collecting_chapter_title = False
    
    lines = content.split('\n')
    # lines = merge_split_chapter_lines(raw_lines)    
    def save_current_article():
        """현재 조를 데이터에 저장합니다."""
        nonlocal article_id
        if current_article and current_content:
            content_text = ' '.join(current_content).strip()
            if content_text:  # 빈 내용은 저장하지 않음
                article_id += 1
                parsed_data.append({
                    'id': article_id,
                    'filename': filename,
                    'last_modified': last_modified,
                    'chapter': f"제{current_chapter}장" if current_chapter else "",
                    'chapter_title': current_chapter_title.strip(),
                    'section': f"제{current_section}절" if current_section else "",
                    'section_title': current_section_title.strip(),
                    'article': current_article,
                    'article_title': current_article_title.strip(),
                    'content': content_text
                })
    
    for i, line in enumerate(lines):
        line = clean_line(line)
        
        # 빈 줄이나 불필요한 줄 건너뛰기
        if not line or line.startswith('-'):
            continue
        
        # 장 제목 수집 중인 경우
        if collecting_chapter_title:
            # 다음 구조 요소(절, 조)가 나오면 장 제목 수집 종료
            if (re.match(r'^제\s*\d+\s*절', line) or 
                re.match(r'^제\s*\d+\s*조', line) or 
                re.match(r'^제\s*\d+\s*장', line) or
                re.match(r'^부\s*칙', line)):
                # 수집된 장 제목 저장
                current_chapter_title = extract_chapter_title(' '.join(chapter_buffer))
                chapter_buffer = []
                collecting_chapter_title = False
                logger.debug(f"장 제목 수집 완료: {current_chapter_title}")
                # 현재 라인을 다시 처리하기 위해 continue하지 않고 아래로 진행
            else:
                chapter_buffer.append(line)
                continue
        
        # 장 패턴 - 더 유연한 패턴 매칭
        chapter_patterns = [
            r"^제\s*(\d+)\s*장\s*(.*)$",
            r"^第\s*(\d+)\s*章\s*(.*)$",  # 한자 표기
            r"^제(\d+)장\s*(.*)$"        # 공백 없는 경우
        ]
        
        chapter_match = None
        for pattern in chapter_patterns:
            chapter_match = re.match(pattern, line)
            if chapter_match:
                break
        
        if chapter_match:
            save_current_article()  # 이전 조 저장
            current_chapter = chapter_match.group(1)
            title_part = chapter_match.group(2).strip()
            
            # 장 제목이 바로 있는 경우
            if title_part and not re.match(r'^제\s*\d+\s*절', title_part):
                current_chapter_title = extract_chapter_title(title_part)
                logger.debug(f"장 제목 즉시 추출: 제{current_chapter}장 - {current_chapter_title}")
            else:
                # 장 제목이 다음 줄에 있을 수 있으므로 수집 모드 시작
                current_chapter_title = ""
                chapter_buffer = [title_part] if title_part else []
                collecting_chapter_title = True
                logger.debug(f"장 제목 수집 모드 시작: 제{current_chapter}장")
            
            # 하위 구조 초기화
            current_section = ""
            current_section_title = ""
            current_article = ""
            current_article_title = ""
            current_content = []
            continue
        
        # 절 패턴
        section_match = re.match(r'^제\s*(\d+)\s*절\s*([^\n<]+?)(?:<.*?>)?$', line)
        if section_match:
            # 장 제목 수집이 진행 중이었다면 강제 완료
            if collecting_chapter_title and chapter_buffer:
                current_chapter_title = extract_chapter_title(' '.join(chapter_buffer))
                chapter_buffer = []
                collecting_chapter_title = False
                logger.debug(f"절 발견으로 장 제목 강제 완료: {current_chapter_title}")
            
            save_current_article()  # 이전 조 저장
            current_section = section_match.group(1)
            current_section_title = section_match.group(2)
            # 조 초기화
            current_article = ""
            current_article_title = ""
            current_content = []
            continue
        
        # 조 패턴 (제목 있는 경우)
        article_match = re.match(r'^제\s*(\d+)\s*조\s*\(([^)]+)\)\s*(.*)$', line)
        if article_match:
            # 장 제목 수집이 진행 중이었다면 강제 완료
            if collecting_chapter_title and chapter_buffer:
                current_chapter_title = extract_chapter_title(' '.join(chapter_buffer))
                chapter_buffer = []
                collecting_chapter_title = False
                logger.debug(f"조 발견으로 장 제목 강제 완료: {current_chapter_title}")
            
            save_current_article()  # 이전 조 저장
            current_article = f"제{article_match.group(1)}조"
            current_article_title = article_match.group(2)
            remaining_text = article_match.group(3).strip()
            current_content = [remaining_text] if remaining_text else []
            continue
        
        # 조 패턴 (제목 없는 경우)
        simple_article_match = re.match(r'^제\s*(\d+)\s*조\s*(.*)$', line)
        if simple_article_match and not re.search(r'\([^)]+\)', line):
            # 장 제목 수집이 진행 중이었다면 강제 완료
            if collecting_chapter_title and chapter_buffer:
                current_chapter_title = extract_chapter_title(' '.join(chapter_buffer))
                chapter_buffer = []
                collecting_chapter_title = False
                logger.debug(f"조 발견으로 장 제목 강제 완료: {current_chapter_title}")
            
            save_current_article()  # 이전 조 저장
            current_article = f"제{simple_article_match.group(1)}조"
            current_article_title = ""
            remaining_text = simple_article_match.group(2).strip()
            current_content = [remaining_text] if remaining_text else []
            continue
        
        # 부칙 패턴
        if re.match(r'^\s*부\s*칙\s*$', line.strip()):
            logger.info(f"부칙 발견됨 - '{filename}' 파일에서 부칙은 저장하지 않습니다.")
            save_current_article()  # 이전 조는 저장
            current_article = ""
            current_article_title = ""
            current_content = []
            continue
        
        # 항목 패턴 (①, 1. 등)
        item_match = re.match(r'^[\s]*([①-⑳]|\d+[.)])\s*(.*)$', line)
        if item_match and current_article:
            current_content.append(line)
            continue
        
        # 일반 내용
        if current_article and line:
            current_content.append(line)
    
    # 루프 종료 후 미완료된 장 제목 처리
    if collecting_chapter_title and chapter_buffer:
        current_chapter_title = extract_chapter_title(' '.join(chapter_buffer))
        logger.debug(f"최종 장 제목 완료: {current_chapter_title}")
    
    # 마지막 조 저장
    save_current_article()
    
    logger.info(f"{filename}에서 {len(parsed_data)}개 조항을 파싱했습니다.")
    return parsed_data

In [87]:
def save_to_csv(data, output_file):
    """데이터를 CSV 파일로 저장합니다."""
    if not data:
        logger.warning("저장할 데이터가 없습니다.")
        return
    
    fieldnames = ['id', 'filename', 'last_modified', 'chapter', 'chapter_title', 
                  'section', 'section_title', 'article', 'article_title', 'content']
    
    try:
        with open(output_file, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        logger.info(f"CSV 파일이 생성되었습니다: {output_file}")
    except Exception as e:
        logger.error(f"CSV 파일 저장 오류: {e}")

def process_all_pdfs_in_dir(base_dir, output_csv):
    """디렉토리 내 모든 PDF 파일을 처리합니다."""
    base_path = Path(base_dir)
    
    if not base_path.exists():
        logger.error(f"디렉토리가 존재하지 않습니다: {base_dir}")
        return
    
    pdf_files = [f for f in os.listdir(base_dir) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        logger.warning(f"PDF 파일을 찾을 수 없습니다: {base_dir}")
        return
    
    all_parsed_data = []
    successful_files = 0
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(base_dir, pdf_file)
        logger.info(f"처리 중: {pdf_file}")
        
        content = extract_text_from_pdf(pdf_path)
        if content is None:
            logger.warning(f"파일 읽기 실패로 건너뜀: {pdf_file}")
            continue
        
        parsed_data = parse_legal_document(pdf_file, content)
        all_parsed_data.extend(parsed_data)
        successful_files += 1
    
    if all_parsed_data:
        save_to_csv(all_parsed_data, output_csv)
        logger.info(f"총 {successful_files}개 파일에서 {len(all_parsed_data)}개 조항을 파싱했습니다.")
    else:
        logger.warning("파싱된 데이터가 없습니다.")

def validate_dependencies():
    """필요한 라이브러리가 설치되어 있는지 확인합니다."""
    try:
        import fitz
        logger.info("PyMuPDF 라이브러리가 정상적으로 로드되었습니다.")
        return True
    except ImportError:
        logger.error("PyMuPDF가 설치되지 않았습니다. 'pip install PyMuPDF'를 실행하세요.")
        return False

def get_parsing_statistics(data):
    """파싱 결과 통계를 출력합니다."""
    if not data:
        return
    
    total_articles = len(data)
    files = set(item['filename'] for item in data)
    chapters = set(item['chapter'] for item in data if item['chapter'])
    sections = set(item['section'] for item in data if item['section'])
    
    logger.info("=== 파싱 통계 ===")
    logger.info(f"처리된 파일 수: {len(files)}")
    logger.info(f"총 조항 수: {total_articles}")
    logger.info(f"장 수: {len(chapters)}")
    logger.info(f"절 수: {len(sections)}")
    
    # 파일별 조항 수
    file_stats = {}
    for item in data:
        filename = item['filename']
        file_stats[filename] = file_stats.get(filename, 0) + 1
    
    logger.info("파일별 조항 수:")
    for filename, count in file_stats.items():
        logger.info(f"  {filename}: {count}개")
    
    # 장 제목이 비어있는 항목 체크
    empty_chapter_titles = [item for item in data if item['chapter'] and not item['chapter_title'].strip()]
    if empty_chapter_titles:
        logger.warning(f"장 제목이 비어있는 조항 {len(empty_chapter_titles)}개 발견")
        for item in empty_chapter_titles[:5]:  # 처음 5개만 표시
            logger.warning(f"  파일: {item['filename']}, 장: {item['chapter']}, 조: {item['article']}")

In [88]:
if __name__ == "__main__":
    if not validate_dependencies():
        exit(1)
    
    base_dir = r"C:\Users\jhwoo\Desktop\SKN_ws\project\SKN13-FINAL-1TEAM\한국방송광고진흥공사\사내규정"
    output_csv = os.path.join(base_dir, "all_parsed_output.csv")
    
    logger.info("PDF 파싱 작업을 시작합니다...")
    
    # 파싱 실행
    base_path = Path(base_dir)
    if base_path.exists():
        pdf_files = [f for f in os.listdir(base_dir) if f.lower().endswith('.pdf')]
        all_parsed_data = []
        
        for pdf_file in pdf_files:
            pdf_path = os.path.join(base_dir, pdf_file)
            logger.info(f"처리 중: {pdf_file}")
            
            content = extract_text_from_pdf(pdf_path)
            if content:
                parsed_data = parse_legal_document(pdf_file, content)
                all_parsed_data.extend(parsed_data)
        
        if all_parsed_data:
            save_to_csv(all_parsed_data, output_csv)
            get_parsing_statistics(all_parsed_data)
        else:
            logger.warning("파싱된 데이터가 없습니다.")
    else:
        logger.error(f"디렉토리가 존재하지 않습니다: {base_dir}")
    
    logger.info("작업이 완료되었습니다.")

2025-08-01 17:48:27,749 - INFO - PyMuPDF 라이브러리가 정상적으로 로드되었습니다.
2025-08-01 17:48:27,751 - INFO - PDF 파싱 작업을 시작합니다...
2025-08-01 17:48:27,753 - INFO - 처리 중: 간접광고 판매대행 약관(외주제작사)(2016년도 9월 제정).pdf
2025-08-01 17:48:27,777 - INFO - 부칙 발견됨 - '간접광고 판매대행 약관(외주제작사)(2016년도 9월 제정).pdf' 파일에서 부칙은 저장하지 않습니다.
2025-08-01 17:48:27,778 - INFO - 간접광고 판매대행 약관(외주제작사)(2016년도 9월 제정).pdf에서 24개 조항을 파싱했습니다.
2025-08-01 17:48:27,779 - INFO - 처리 중: 감사규정(2025년도 6월 개정).pdf
2025-08-01 17:48:27,999 - INFO - 감사규정(2025년도 6월 개정).pdf에서 57개 조항을 파싱했습니다.
2025-08-01 17:48:28,001 - INFO - 처리 중: 개인정보보호지침(2024년도 5월 개정).pdf
2025-08-01 17:48:28,318 - INFO - 개인정보보호지침(2024년도 5월 개정).pdf에서 59개 조항을 파싱했습니다.
2025-08-01 17:48:28,318 - INFO - 처리 중: 경력개발제도 및 전문직무제 운영지침(2025년도 3월 개정).pdf
2025-08-01 17:48:28,432 - INFO - 경력개발제도 및 전문직무제 운영지침(2025년도 3월 개정).pdf에서 20개 조항을 파싱했습니다.
2025-08-01 17:48:28,432 - INFO - 처리 중: 경영공시 관리지침(2020년도 12월 개정).pdf
2025-08-01 17:48:28,506 - INFO - 경영공시 관리지침(2020년도 12월 개정).pdf에서 16개 조항을 파싱했습니다.
2025-08-01 17:48:28,51